In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [12]:
number_of_eqps_values = 21
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'processed_data/vm_1_bingo_format.txt'
pre_data = np.loadtxt(dataloc)
#print(pre_data)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(eqpses)
#print(training_data)

def mapping_(x, return_error=True):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    mapping = lambda X_0: (array([[x[0], x[1], x[2]],
            [x[3], x[4], x[5]],
            [x[6], x[7], x[8]]])) * ((array(x[9]) + X_0 + array(x[9]) + X_0) * (X_0)) + array([[x[10], x[11], x[12] ],        
            [ x[13],  x[14] , x[15]],
            [ x[16], x[17],  x[18]]])
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
    #     [0 , x[6], 0],
    #     [0, 0, x[7]]]))
    # ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],
    #    [-0.8733965 , -1.02045703, -0.37967495],
    #    [-0.86253049, -1.02066836, -0.38872487]]))
    #mapping = lambda X_0: ((x[0]*X_0 + x[1]) )*((x[2]*X_0 + x[3]) ) * np.eye(3)
    
    # (array([[x[2], x[3], x[4] ],  
    #     [x[3] , x[6], x[5]],
    #     [x[4], x[5], x[7]]]))
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = np.transpose(mapping_matrices,(0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        back_calculated_yield_stresses.append(np.transpose(training_data[i]) @ P_mapped[i] @ training_data[i])
        #back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])
        #print(P_mapped[i] @ training_data[i])
    #print(back_calculated_yield_stresses)
    back_calculated_yield_stresses = np.array(back_calculated_yield_stresses)
    Y_0 = 10
    H = 990
    y = np.array([ (eqps_*H + Y_0)**2 for eqps_ in eqpses ])
    #print(back_calculated_yield_stresses.shape, y.shape)
    error = y - back_calculated_yield_stresses
    #error = training_data - np.array(back_calculated_yield_stresses)
    #print(error)
    if return_error:
        return error.flatten()
    else:
        return back_calculated_yield_stresses, y, mapping_matrices


# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


In [24]:
x0 = [-0.32419465, -0.59102337, -0.18063179, -0.38121066, -0.61755865, -0.23576606, -0.28240491, -0.54735189, -0.16744104, -0.3588380, 
      -0.42357774, -0.04148228,  0.5943027, 0.57707663,  0.0577839 , -0.40190232, 0.64225345, -0.97251048,  0.56466269
]*2
#x0 = [ 1, 1, 1, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]

mapping_(x0)
#results = optimize.minimize(mapping_, )

array([-8.7375534803868504e-05, -2.6246382896033538e-04,
       -4.3504089398993528e-04, -5.5242690814338857e-04,
       -5.9115963222211576e-04, -5.4948007709754165e-04,
       -4.4064169469493208e-04, -2.8704310534521937e-04,
       -1.1518433711898979e-04,  4.8553376473137178e-05,
        1.8130430726159830e-04,  2.6629140302247833e-04,
        2.9457591881509870e-04,  2.6598351905704476e-04,
        1.8920688671641983e-04,  8.1084741395898163e-05,
       -3.4942593629239127e-05, -1.3120117364451289e-04,
       -1.7897506768349558e-04, -1.5252128650899976e-04,
       -3.3908407203853130e-05, -8.7375534803868504e-05,
       -2.6246382896033538e-04, -4.3504089410362212e-04,
       -5.5242690791601490e-04, -5.9115963222211576e-04,
       -5.4948007618804695e-04, -4.4064169287594268e-04,
       -2.8704310170724057e-04, -1.1518433802848449e-04,
        4.8553380111115985e-05,  1.8130431271856651e-04,
        2.6629140302247833e-04,  2.9457591881509870e-04,
        2.6598352633300237e-04,

In [25]:
results = optimize.root(mapping_, x0=x0, method='lm', tol=1e-16, options={"ftol": 1e-7, "gtol": 1e-16, "maxiter": 100000})
results

 message: Both actual and predicted relative reductions in the sum of squares
            are at most 0.000000
 success: True
  status: 1
     fun: [-2.517e-05 -6.805e-05 ...  9.431e-05 -9.379e-05]
       x: [-3.242e-01 -5.910e-01 ... -9.725e-01  5.647e-01]
   cov_x: None
  method: lm
    nfev: 572
    fjac: [[-5.999e+05  7.099e+03 ...  8.005e-02  8.824e-02]
           [-2.896e+05  5.126e+05 ...  3.980e-02  4.388e-02]
           ...
           [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]]
    ipvt: [18 11 ... 37 38]
     qtf: [ 2.757e-07  1.225e-06 ... -8.105e-07  1.536e-05]

In [26]:
np.set_printoptions(threshold=sys.maxsize)
error_array = mapping_(results.x)
print(error_array, '\nMax:', np.max(error_array))

[-2.5168252221874354e-05 -6.8053754887387186e-05 -9.9054299084855302e-05
 -1.0638827734510414e-04 -9.0390661625860957e-05 -5.8628921124181943e-05
 -2.1842413843842223e-05  9.2947311713942327e-06  2.6587270440359134e-05
  2.5908984753186814e-05  7.9693836596561596e-06 -2.1743095203419216e-05
 -5.3841366025153548e-05 -7.6743486715713516e-05 -7.9199835454346612e-05
 -5.3643896535504609e-05 -3.6640994949266315e-07  6.7486933403415605e-05
  1.1809427815023810e-04  9.4313036242965609e-05 -9.3788243248127401e-05
 -2.5168213738879786e-05 -6.8053611755658494e-05 -9.9053996791553800e-05
 -1.0638777075655526e-04 -9.0389914021216100e-05 -5.8627900216379203e-05
 -2.1841091438545845e-05  9.2963819042779505e-06  2.6589274966681842e-05
  2.5911380362231284e-05  7.9722067312104627e-06 -2.1739808289567009e-05
 -5.3837560699321330e-05 -7.6739084761356935e-05 -7.9194782301783562e-05
 -5.3638079407392070e-05 -3.5973789636045694e-07  6.7494620452634990e-05
  1.1810311116278172e-04  9.4323200755752623e-05 -9

In [27]:
np.set_printoptions(precision=32)
results.x

array([-0.324190879399528  , -0.5910265708699819 , -0.18063603062696332,
       -0.38121381734569654, -0.6175628700881645 , -0.23576228096254312,
       -0.28240916917730036, -0.5473481732181913 , -0.16744427128767322,
       -0.2630318047470945 , -0.4235775835834732 , -0.04148242995369289,
        0.5943025397254625 ,  0.5770764819925227 ,  0.05778373947936392,
       -0.4019021627391064 ,  0.6422532886726127 , -0.9725103257468183 ,
        0.564662540091747  , -0.32419465         , -0.59102337         ,
       -0.18063179         , -0.38121066         , -0.61755865         ,
       -0.23576606         , -0.28240491         , -0.54735189         ,
       -0.16744104         , -0.358838           , -0.42357774         ,
       -0.04148228         ,  0.5943027          ,  0.57707663         ,
        0.0577839          , -0.40190232         ,  0.64225345         ,
       -0.97251048         ,  0.56466269         ])

In [28]:
predicted_YS, actual_Y, mappings = mapping_(results.x, return_error=False)

In [29]:
predicted_YS

array([  100.00002516825222,   396.0100680537548 ,   888.0400990542992 ,
        1576.0901063882775 ,  2460.160090390662  ,  3540.250058628921  ,
        4816.360021842414  ,  6288.48999070527   ,  7956.63997341273   ,
        9820.809974091015  , 11880.999992030616  , 14137.210021743096  ,
       16589.44005384137   , 19237.690076743493  , 22081.960079199842  ,
       25122.250053643897  , 28358.56000036641   , 31790.88993251307   ,
       35419.23988190572   , 39243.609905686964  , 43264.00009378824   ,
         100.00002516821374,   396.0100680536117 ,   888.0400990539969 ,
        1576.090106387771  ,  2460.1600903899143 ,  3540.2500586279    ,
        4816.360021841092  ,  6288.48999070362   ,  7956.639973410725  ,
        9820.80997408862   , 11880.999992027793  , 14137.21002173981   ,
       16589.440053837563  , 19237.69007673909   , 22081.96007919479   ,
       25122.25005363808   , 28358.56000035974   , 31790.889932505383  ,
       35419.23988189689   , 39243.6099056768    , 

In [30]:
actual_Y

array([  100.              ,   396.00999999999993,   888.0400000000001 ,
        1576.0900000000001 ,  2460.1600000000003 ,  3540.25            ,
        4816.360000000001  ,  6288.490000000002  ,  7956.64            ,
        9820.81            , 11881.              , 14137.210000000001  ,
       16589.440000000002  , 19237.690000000006  , 22081.960000000006  ,
       25122.25            , 28358.56            , 31790.890000000003  ,
       35419.24            , 39243.61            , 43264.              ,
         100.              ,   396.00999999999993,   888.0400000000001 ,
        1576.0900000000001 ,  2460.1600000000003 ,  3540.25            ,
        4816.360000000001  ,  6288.490000000002  ,  7956.64            ,
        9820.81            , 11881.              , 14137.210000000001  ,
       16589.440000000002  , 19237.690000000006  , 22081.960000000006  ,
       25122.25            , 28358.56            , 31790.890000000003  ,
       35419.24            , 39243.61            , 

In [31]:
mappings

array([[[-0.4235775835834732  , -0.04148242995369289 ,
          0.5943025397254625  ],
        [ 0.5770764819925227  ,  0.05778373947936392 ,
         -0.4019021627391064  ],
        [ 0.6422532886726127  , -0.9725103257468183  ,
          0.564662540091747   ]],

       [[-0.421936971517533   , -0.03849145955607853 ,
          0.5952166729421003  ],
        [ 0.5790056663964729  ,  0.06090900043062799 ,
         -0.4007090556302415  ],
        [ 0.6436824587076939  , -0.9697403958249298  ,
          0.5655099146149166  ]],

       [[-0.4204260358033526  , -0.03573689978681216 ,
          0.5960585517464875  ],
        [ 0.5807823652734848  ,  0.06378723623385679 ,
         -0.39961025343376166 ],
        [ 0.6449986650751042  , -0.9671894051723285  ,
          0.5662903114295712  ]],

       [[-0.419044776440932   , -0.03321875064589379 ,
          0.5968281761386237  ],
        [ 0.5824065786235585  ,  0.06641844688905034 ,
         -0.3986057561496668  ],
        [ 0.64620190777484